In [ ]:
# import Flask class from the flask module
from flask import Flask
from flask import request
import pandas as pd
import numpy as np
import os
import sklearn.pipeline
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest
from sklearn.svm import SVC
import pickle
from io import StringIO


def root_directory():
    current_path = os.path.abspath('')
    return os.path.abspath(os.path.join(current_path))
def data_directory():
    return os.path.join(root_directory(), "data")

def load_train_set():
    #Loading a hdf5 file is much much faster
    in_file = os.path.join(data_directory(), "final",  "train.csv")
    return pd.read_csv(in_file)
def load_test_set(str):
    data = StringIO(str)
    in_file = str
    return pd.read_csv(data)

def simple_model_evaluation(str):
    select = SelectKBest(k=20)
    train =load_train_set()
    test = load_test_set(str)
    target = 'condition'
    hrv_features = list(train)
    hrv_features = [x for x in hrv_features if x not in [target]]
    X_train= train[hrv_features]
    y_train= train[target]
    X_test = test[hrv_features]
    y_test = test[target]
        
    # Normalize the attribute values to mean=0 and variance=1
    from sklearn.preprocessing import StandardScaler
    scaler = StandardScaler()
    scaler.fit(X_train)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)
    

    
    y_pred = svmModel.predict(X_test)
    
    #print(sklearn.metrics.accuracy_score(y_test, y_pred))
    #print(sklearn.metrics.classification_report(y_test, y_pred))
    return sklearn.metrics.classification_report(y_test, y_pred)

# Create Flask object to run
app = Flask(__name__)


@app.route('/')
def home():
    #text = simple_model_evaluation()
    return simple_model_evaluation()

@app.route("/debug", methods=["POST"])
def debug():
    text = request.form["csv"]
    title = request.form["title"]
    return simple_model_evaluation(text)+title  

# Load the pre-trained and persisted SVM model
# Note: The model will be loaded only once at the start of the server
def load_model():
    global svmModel
    
    svmFile = open('models/SVMModel.pckl', 'rb')
    svmModel = pickle.load(svmFile)
    svmFile.close()

if __name__ == "__main__":
    print("**Starting Server...")
    
    # Call function that loads Model
    load_model()
    
    
    # Run Server
    app.run(host="192.168.0.17")

**Starting Server...
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://192.168.0.17:5000/ (Press CTRL+C to quit)


In [9]:
# import Flask class from the flask module
from flask import Flask, request

import numpy as np
import pickle


# Create Flask object to run
app = Flask(__name__)


@app.route('/')
def home():
    return "Hi, Welcome to Flask!!"

@app.route('/predict')
def predict():

    # Get values from browser
    sepLen = request.args['sepal_length']
    sepWid = request.args['sepal_width']
    petLen = request.args['petal_length']
    petWid = request.args['petal_width']

    testData = np.array([sepLen, sepWid, petLen, petWid]).reshape(1,4)
    class_prediced = int(svmIrisModel.predict(testData)[0])
    output = "Predicted Iris Class: " + str(class_prediced)

    return (output)

# Load the pre-trained and persisted SVM model
# Note: The model will be loaded only once at the start of the server
def load_model():
    global svmModel
    
    svmFile = open('models/SVMModel.pckl', 'rb')
    svmModel = pickle.load(svmFile)
    svmFile.close()

if __name__ == "__main__":
    print("**Starting Server...")
    
    # Call function that loads Model
    load_model()
    
    
    # Run Server
    app.run()

**Starting Server...
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


In [12]:
import pandas as pd
import numpy as np
import os
import sklearn.pipeline
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest
from sklearn.svm import SVC
import pickle


def root_directory():
    current_path = os.path.abspath('')
    return os.path.abspath(os.path.join(current_path))
def data_directory():
    return os.path.join(root_directory(), "data")

def load_train_set():
    #Loading a hdf5 file is much much faster
    in_file = os.path.join(data_directory(), "final",  "train.csv")
    return pd.read_csv(in_file)
def load_test_set():
    #Loading a hdf5 file is much much faster
    in_file = os.path.join(data_directory(), "final",  "test.csv")
    return pd.read_csv(in_file)

def simple_model_evaluation():
    select = SelectKBest(k=20)
    train =load_train_set()
    test = load_test_set()
    target = 'condition'
    hrv_features = list(train)
    hrv_features = [x for x in hrv_features if x not in [target]]
    X_train= train[hrv_features]
    y_train= train[target]
    X_test = test[hrv_features]
    y_test = test[target]
        
    # Normalize the attribute values to mean=0 and variance=1
    from sklearn.preprocessing import StandardScaler
    scaler = StandardScaler()
    scaler.fit(X_train)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)
    

    steps = [('feature_selection', select),
           ('model', SVC(C=80, gamma=0.1,
                         kernel='rbf'))]
    print(steps)
    pipeline = sklearn.pipeline.Pipeline(steps)
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    pickle.dump(pipeline, open('model.pckl','wb'))
    print("-------------------------------------------------------")
    print(sklearn.metrics.classification_report(y_test, y_pred))
    print()
    print()
    
simple_model_evaluation()

[('feature_selection', SelectKBest(k=20)), ('model', SVC(C=80, gamma=0.1))]
-------------------------------------------------------
              precision    recall  f1-score   support

   no stress       0.95      0.92      0.93       108
    stressed       0.91      0.95      0.93        92

    accuracy                           0.93       200
   macro avg       0.93      0.93      0.93       200
weighted avg       0.93      0.93      0.93       200





C:\Users\nss\anaconda3\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:114: UserWarning: Features [34] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx,
C:\Users\nss\anaconda3\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:116: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw
